In [8]:
import requests
import json


In [9]:
response = requests.get('https://static.openfoodfacts.org/data/taxonomies/additives.json', stream=True)

if response.status_code == 200:
    with open('../additives.json', 'wb') as file :
        for chunk in response.iter_content(chunk_size=1000):
            file.write(chunk)
else:
    print(f"Une erreur s'est produite lors du télechargement du fichier")

In [10]:
from selenium import webdriver

from selenium.webdriver.chrome.options import Options

from bs4 import BeautifulSoup
import time


options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)  


driver.get("https://www.additifs-alimentaires.net/additifs.php")

time.sleep(10)  

html = driver.page_source

soup = BeautifulSoup(html, "html.parser")

table = soup.select_one("table.tpi tbody")

additives_list = []

for row in table.find_all('tr'):
    additive_dict = {
        'additive_code': row.select_one('td.colCode').get_text(strip=True),
        'names': row.select_one('td.colNom').get_text(strip=True).split(', '),
        'danger': row.select_one('td.colDanger').get_text(strip=True),
    }
    print(additive_dict)
    # print(row.get_text(separator="\n", strip=True))
    print("\n")
    additives_list.append(additive_dict)
# print(table.get_text(separator="\n", strip=True))
print(len(additives_list))
driver.quit()


{'additive_code': 'E100', 'names': ['Curcumines', 'Jaune de curcumine'], 'danger': 'Ne pas abuser'}


{'additive_code': 'E101', 'names': ['Riboflavine', "Riboflavine-5'-phosphate", 'Vitamine B2'], 'danger': 'Ne pas abuser'}


{'additive_code': 'E102', 'names': ['Tartrazine'], 'danger': 'Très toxique'}


{'additive_code': 'E104', 'names': ['Jaune de quinoléine', 'Jaune de quinoline'], 'danger': 'Très toxique'}


{'additive_code': 'E105', 'names': ['Jaune solide'], 'danger': 'Très toxique'}


{'additive_code': 'E107', 'names': ['Jaune 2G'], 'danger': 'Toxique'}


{'additive_code': 'E110', 'names': ['Jaune orangé S', 'Jaune soleil FCF'], 'danger': 'Très toxique'}


{'additive_code': 'E111', 'names': ['Orange GGN', 'Alpha-naphthol orange'], 'danger': 'Très toxique'}


{'additive_code': 'E120', 'names': ['Cochenille', 'Acide Carminique', 'Carmins'], 'danger': 'Douteux'}


{'additive_code': 'E121', 'names': ['Rouge citrus no. 2 (ex- Orseille', 'Orcéine)'], 'danger': 'Très toxique'}


{'addit

In [16]:
options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)  

complete_additives_list = []
for additive in additives_list:
    code = additive['additive_code']

    url = f"https://www.additifs-alimentaires.net/{code}.php#toxic"
    
    try: 
        driver.get(url)
        time.sleep(1)
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        texts = soup.find_all("td", class_="colonne2")

        description = texts[0].get_text(strip=True)
        description_avancee = texts[2].get_text(strip=True)
        remarques = texts[4].get_text(strip=True)
        additive['description'] = description
        additive['description_avancee'] = description_avancee
        additive['remarques'] = remarques
        print(additive)
        complete_additives_list.append(additive)
    except Exception as e:
        print(f"Erreur lors de la récupération de la page pour {code}: {e}")
        continue


    

{'additive_code': 'E100', 'names': ['Curcumines', 'Jaune de curcumine'], 'danger': 'Ne pas abuser', 'description': "La curcumine est le pigment principal jaune-orange de la plante herbacée curcuma (Curcuma longa, ouSafran des Indes)  [21a]. Elle peut être isolée par distillation à la vapeur, ou extraite parsolvantsd'extraction chimiques [24,9], comme c'est le cas dans l'Union Européenne [15f].", 'description_avancee': 'La curcumine E100 est généralement décrite sans effet secondaire connu [1,2,4,9,9\',9",10,22,28] sauf réactions allergiques rares (?dèmes de Quincke, urticaires aigües, etc.) [9,22,23,29].Mais la curcumine naturelle, connue pour ses effets bénéfiques, ne doit pas être confondue avec la curcumine du Codex alimentarius ou E100 qui, bien que d\'origine naturelle peut contenir jusque 30% d\'aluminium (E173), et des traces de solvants dans ses résidus de productionVoir sec°. Description > Définitions officielles > Résidus.Cette curcumine de l\'alimentation transformée est dan

In [18]:
import json

with open('../additives_complet_data.json', 'wb') as file : 
        file.write(json.dumps(complete_additives_list, ensure_ascii=False, indent=4).encode('utf-8'))
print("Fichier JSON créé avec succès.")

Fichier JSON créé avec succès.


In [ ]:


from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017")
db = client["Nutriwiser"]
collection = db["additives"]

for additive in complete_additives_list:
    document = {
        'additive_code': additive['additive_code'],
        'names': additive['names'],
        'danger': additive['danger']
    }
    print(document)


    collection.insert_one(document)



{'additive_code': 'E100', 'names': ['Curcumines', 'Jaune de curcumine'], 'danger': 'Ne pas abuser'}
{'additive_code': 'E101', 'names': ['Riboflavine', "Riboflavine-5'-phosphate", 'Vitamine B2'], 'danger': 'Ne pas abuser'}
{'additive_code': 'E102', 'names': ['Tartrazine'], 'danger': 'Très toxique'}
{'additive_code': 'E104', 'names': ['Jaune de quinoléine', 'Jaune de quinoline'], 'danger': 'Très toxique'}
{'additive_code': 'E105', 'names': ['Jaune solide'], 'danger': 'Très toxique'}
{'additive_code': 'E107', 'names': ['Jaune 2G'], 'danger': 'Toxique'}
{'additive_code': 'E110', 'names': ['Jaune orangé S', 'Jaune soleil FCF'], 'danger': 'Très toxique'}
{'additive_code': 'E111', 'names': ['Orange GGN', 'Alpha-naphthol orange'], 'danger': 'Très toxique'}
{'additive_code': 'E120', 'names': ['Cochenille', 'Acide Carminique', 'Carmins'], 'danger': 'Douteux'}
{'additive_code': 'E121', 'names': ['Rouge citrus no. 2 (ex- Orseille', 'Orcéine)'], 'danger': 'Très toxique'}
{'additive_code': 'E122', '